# Facility Information
Extract the subset of facility information for all NPI on claims for this analysis

**Script**
* [scripts/de/facility_info.ipynb](./scripts/de/facility_info.ipynb)

**Prior Script(s)**
* [scripts/coh/coh_basic_prov.ipynb](./scripts/coh/coh_basic_prov.ipynb)

**Parameters**
* NONE

**Input**
* `coh_npi`

**Output**  
* `de_facility_to_parent`
* `de_facility_hb_npi_link`

**Review**
* [scripts/de/facility_info.html](./scripts/de/facility_info.html)

In [ ]:
#Import libraries for this notebook
import pandas as pd  
from drg_connect import Snowflake
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load connection variables to connect_dict
with open('../../out/conn/connect_dict.pickle', 'rb') as handle:
    connect_dict = pickle.load(handle)

#Create Eegine to connect to snowflake
snow = Snowflake(role=connect_dict['role'],
                 warehouse=connect_dict['warehouse'],
                 database=connect_dict['database'],
                 schema=connect_dict['schema'])

#Finish engine setup
engine = snow.engine
%load_ext sql_magic
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications


# Extract Data

In [ ]:
%%read_sql
--Create raven diagnosis table
DROP TABLE IF EXISTS de_facility_to_parent;
CREATE TRANSIENT TABLE de_facility_to_parent AS 
  SELECT DISTINCT fac.*
    FROM ref_db.analytics.facility_to_parent fac
         JOIN rwd_db.rwd.healthbase_entitylookup ent
           ON ent.healthbase_id = fac.facility_hb_id
   WHERE ent.identifier_type = 'NPI'
         AND ent.identifier IN (SELECT npi 
                                  FROM coh_npi); 

In [ ]:
%%read_sql
SELECT Count(*) AS row_cnt,
       Count(Distinct facility_hb_id) AS facility_cnt
  FROM de_facility_to_parent

In [ ]:
%%read_sql
--Create subset of rwd_db.rwd.healthbase_entitylookup to map NPIs to Healthbase Ids
DROP TABLE IF EXISTS de_facility_hb_npi_link;
CREATE TRANSIENT TABLE de_facility_hb_npi_link AS
    SELECT healthbase_id AS facility_hb_id,
           identifier    AS npi,
           label
      FROM rwd_db.rwd.healthbase_entitylookup
     WHERE identifier_type = 'NPI';

In [ ]:
%%read_sql
--Review Counts of the npi-healthbase id link
SELECT Count(*) AS row_cnt,
       Count(Distinct facility_hb_id) AS hb_cnt,
       Count(Distinct npi) AS npi_cnt
  FROM de_facility_hb_npi_link;